In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocess as mp
import time
from helpers.GPTS_Kernels import RBFKernel
import helpers.GP_opt_sims as GP

In [89]:
def genFunc(func_info, beta):
    if(func_info[0] == 0):
        _, m, o, l = func_info
        truef = lambda x: o * np.exp(-(x-m)*(x-m)/l)
        return GP.GPFunction(truef, 1/beta, [0, 1], [m, o])
    
def GP_sim_wrapper(info, print_upd = True): 
    import numpy as np
    
    idnum, task = info
    T, func_info, AR_type, data_amt, data_dist, num_samples, gpr_data = task
    len_scale, beta = gpr_data

    kernel = RBFKernel(len_scale)
    func = genFunc(func_info, beta)

    PERC_FREQ = 0.1
    print_intvl = int(num_samples*PERC_FREQ/100)
    print_intvl = print_intvl if print_intvl > 0 else 1

    regret_vecs = [np.zeros((num_samples, T)) for _ in range(2)]
    NUM_BUCKETS = 50
    data_rem = np.zeros(NUM_BUCKETS)
    buckets = np.linspace(func.x_min, func.x_max, NUM_BUCKETS + 1)
    for i in range(num_samples):
        #generate data: TODO-non uniform
        data_X = np.random.uniform(func.x_min, func.x_max, size = (data_amt, 1))
        data_Y = np.array([func.sample(p) for p in data_X])

        fs_rew = GP.FS(T, func, data_X, data_Y, kernel, beta)

        if(AR_type[0] == 0): #level set
            ar_rew, data_left = GP.AR_lvl_set(T, func, data_X, data_Y, kernel, beta, AR_type[1])
        elif(AR_type[0] == 1): #fixed data radius
            ar_rew, data_left = GP.AR_rad(T, func, data_X, data_Y, kernel, beta, AR_type[1])
        else:
            pass      
    
        regret_vecs[0][i] = np.cumsum(func.opt - np.array(fs_rew))
        regret_vecs[1][i] = np.cumsum(func.opt - np.array(ar_rew))
        hist, _ = np.histogram(data_left, buckets)
        data_rem = data_rem + hist
        
        if(print_upd and ((i+1)%(print_intvl) == 0)):
            whitespace = " " * 20
            print("Task #" +str(idnum) + ":" + str(task) + " is " + str((i+1)/num_samples*100) + "% completed." + whitespace , end='\r')
    
    avgs = np.mean(regret_vecs, axis=1)
    lower_CBs = avgs-2*np.std(regret_vecs,axis=1)/np.sqrt(num_samples)
    upper_CBs = avgs+2*np.std(regret_vecs,axis=1)/np.sqrt(num_samples)
        
    fs_row = [str(task)] + list(avgs[0]) + list(lower_CBs[0]) + list(upper_CBs[0])
    ar_row = [str(task)] + list(avgs[1]) + list(lower_CBs[1]) + list(upper_CBs[1])

    df_cols = ["Params"]
    c = list(range(1, T + 1))
    df_cols = df_cols + c + ["low_"+str(n) for n in c] + ["high_"+str(n) for n in c]

    fs_df= pd.DataFrame(columns = df_cols)
    fs_df.loc[0] = fs_row
    ar_df= pd.DataFrame(columns = df_cols)
    ar_df.loc[0] = ar_row
    fs_df.to_csv("temp/fs_task"+str(idnum)+".csv")
    ar_df.to_csv("temp/ar_task"+str(idnum)+".csv")

In [90]:
#fixed hyperparams
beta = 10
len_scale = 10

In [91]:
#simulation params
T = 5000 #simulation horizon
num_samples = 20000
#different ways of data distribution across sample space
data_dists = [0] #0-uniform,
functions = [[0, 0.5, 1, 1/20]] #0-single peak, 1-bimodal, 2-?
data_amt_list = np.arange(0, 155, 15) #total number of data points (16)
AR_types = [[0, x] for x in [0.25, 0.5, 1, 2, 4]] + [[1, x] for x in [0.05, 0.1, 0.2, 0.4]] #0-level set, 1-bandwidth

#generate tasks
tasks = [[T, func, AR_type, data_amt, data_dist, num_samples, [len_scale, beta]] 
         for func in functions for AR_type in AR_types for data_amt in data_amt_list for data_dist in data_dists]
info = [(i, tasks[i]) for i in range(len(tasks))]

In [92]:
#simulation params
T = 200 #simulation horizon
num_samples = 10
#different ways of data distribution across sample space
data_dists = [0] #0-uniform,
functions = [[0, 0.5, 1, 1/20]] #0-single peak, 1-bimodal, 2-?
data_amt_list = [25] #total number of data points (16)
AR_types = [[0, 2]] #0-level set, 1-bandwidth

#generate tasks
tasks = [[T, func, AR_type, data_amt, data_dist, num_samples, [len_scale, beta]] 
         for func in functions for AR_type in AR_types for data_amt in data_amt_list for data_dist in data_dists]
info = [(i, tasks[i]) for i in range(len(tasks))]

In [94]:
GP_sim_wrapper(info[0], print_upd = False)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]